
### This is the graph of fuel spend.  
The database work was done in __/media/lnr-ai/christo/github_repos/graphing/project_56101.cql__ 

In this work we aim to achieve the following:
Merchant rank each of the fuel station communities where the member count is greater than 4.

Load libraries:

In [2]:
import pandas as pd
from py2neo import Graph
graph = Graph("bolt://localhost:7687", user='neo4j', password='NewPassword')
import time
import glob
import os

### Merchant ranking

The following needs to run only once. It creates a parameter  called __merchant_56101_rank__ on each of the fuel station merchant nodes that belongs to a community of fuel stations where the member counts is more than some minimum.  

This portion of the script only needs to run only once

Define neo4j query to extract all fuel station communities as created in /media/lnr-ai/christo/github_repos/graphing/project_56101.cql.  Draw the community number and the member count:

In [2]:
neo4j_query="""
MATCH (m:Merchant {subclass_id:56101})
RETURN COUNT(m.merchant_56101_community) as membercount, m.merchant_56101_community as merchant_56101_community
ORDER BY membercount ASC;
"""
merchant_rank_df=graph.run(neo4j_query).to_data_frame()

View this dataframe:

In [3]:
merchant_rank_df

,membercount,merchant_56101_community
0,1,2
1,1,8992
2,1,5
3,1,8993
4,1,7
...,...,...
4483,285,3033
4484,312,1565
4485,439,3312
4486,592,2134


Limit this dataframe to communities with more than 4 members:

In [4]:
merchant_rank_df=merchant_rank_df[merchant_rank_df['membercount']>4].copy()

Create a unique list of all community members:

In [5]:
merchant_56101_community_list=list(set(merchant_rank_df['merchant_56101_community']))

Save the above dataframe to a .csv file:

In [6]:
merchant_rank_df_copy=merchant_rank_df.copy()
merchant_rank_df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_community_df',index=False)

In [7]:
merchant_rank_df_copy

,membercount,merchant_56101_community
4460,12,2937
4461,19,1675
4462,20,2245
4463,20,1521
4464,27,1282
4465,56,1668
4466,62,1833
4467,77,728
4468,77,3516
4469,79,1502


Set the first element of merchant_56101_community_list equal to merchant_56101_community:

In [9]:
merchant_56101_community=merchant_56101_community_list[0]

Create a subgraph of all merchants in the merchant_56101_community; we will drop this graph in the iteration that follows.

In [13]:
# Create a subgraph of all merchants in merchant_56101_community:
print('Create subgraph on community {merchant_56101_community}:'.format(merchant_56101_community=merchant_56101_community))
neo4j_query="""CALL gds.graph.create.cypher(
    'merchant_56101_graph',
    'MATCH (m0:Merchant {{subclass_id:56101}}) 
    WHERE m0.merchant_56101_community = {merchant_56101_community} RETURN id(m0) AS id, 
    m0.totaltransactionamount as totaltransactionamount, m0.totaltransactioncount as totaltransactioncount',
    'MATCH (m0:Merchant)-[rel:MERCHANT_FEET_LINK_56101]->(m1:Merchant) 
    WHERE rel.count > 1 AND 
    m0.merchant_56101_community = {merchant_56101_community} AND 
    m1.merchant_56101_community = {merchant_56101_community}
    RETURN id(m0) AS source, id(m1) AS target, rel.count as count'
)
YIELD graphName, nodeCount, relationshipCount, createMillis;
""".format(merchant_56101_community=merchant_56101_community)
graph.run(neo4j_query).data()


Create subgraph on community 1565:


[{'graphName': 'merchant_56101_graph',
  'nodeCount': 312,
  'relationshipCount': 0,
  'createMillis': 1823}]

In the following iteration we iterate over the contents of the merchant_56101_community_list list.

In [11]:
merchant_rank_df=pd.DataFrame()
# merchant_56101_community=merchant_56101_community_list[0]
A=merchant_56101_community_list.copy()
counter=0
for merchant_56101_community in merchant_56101_community_list:
    counter+=1
    print('Doing community {merchant_56101_community}; {counter} of {countlen}'.format(merchant_56101_community=merchant_56101_community, counter=counter,countlen=len(merchant_56101_community_list)))
    print('Drop subgraph:')
    neo4j_query="""CALL gds.graph.drop('merchant_56101_graph') YIELD graphName;"""
    graph.run(neo4j_query).data()
# Create a subgraph of all merchants in merchant_56101_community:
    print('Create subgraph on community {merchant_56101_community}:'.format(merchant_56101_community=merchant_56101_community))
    neo4j_query="""CALL gds.graph.create.cypher(
        'merchant_56101_graph',
        'MATCH (m0:Merchant {{subclass_id:56101}}) 
        WHERE m0.merchant_56101_community = {merchant_56101_community} RETURN id(m0) AS id, 
        m0.totaltransactionamount as totaltransactionamount, m0.totaltransactioncount as totaltransactioncount',
        'MATCH (m0:Merchant)-[rel:MERCHANT_FEET_LINK]->(m1:Merchant) 
        WHERE rel.count > 1 AND 
        m0.merchant_56101_community = {merchant_56101_community} AND 
        m1.merchant_56101_community = {merchant_56101_community}
        RETURN id(m0) AS source, id(m1) AS target, rel.count as count'
    )
    YIELD graphName, nodeCount, relationshipCount, createMillis;
    """.format(merchant_56101_community=merchant_56101_community)
    graph.run(neo4j_query).data()
# Write the MerchantRank on community nodes.  The rank belongs to the community of merchants as defined by the sub graph created above:
    print('MerchantRank community members:')
    neo4j_query="""
    CALL gds.pageRank.write('merchant_56101_graph', {
      maxIterations: 20,
      dampingFactor: 0.85,
      writeProperty: 'merchant_56101_rank',
      relationshipWeightProperty: 'count'
    })
    YIELD nodePropertiesWritten AS writtenProperties, ranIterations;"""
    graph.run(neo4j_query).data()
    print('Extract community members with merchant rank: ')
    neo4j_query="""
    MATCH (m:Merchant {{subclass_id:56101}}) WHERE m.merchant_56101_community = {merchant_56101_community}
    RETURN m.franchisename as franchisename, m.companyname as companyname,  m.merchant_56101_rank as MerchantRank, {merchant_56101_community} as merchant_56101_community 
    ORDER BY MerchantRank DESC;
    """.format(merchant_56101_community=merchant_56101_community)
    df=graph.run(neo4j_query).to_data_frame()

    df['rankID']=range(1,df.shape[0]+1)
    print('Append extracted merchant community to a result dataframe:')    
    merchant_rank_df=merchant_rank_df.append(df,ignore_index=True)
    A.pop(A.index(merchant_56101_community))
    print('====================================================================================================')

Doing community 1282; 1 of 28
Drop subgraph:
Create subgraph on community 1282:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 1668; 2 of 28
Drop subgraph:
Create subgraph on community 1668:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 2186; 3 of 28
Drop subgraph:
Create subgraph on community 2186:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 1675; 4 of 28
Drop subgraph:
Create subgraph on community 1675:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 2066; 5 of 28
Drop subgraph:
Create subgraph on community 2066:
MerchantRank community members:
Extract communit

ClientError: ProcedureCallFailed: Failed to invoke procedure `gds.pageRank.write`: Caused by: org.neo4j.kernel.availability.UnavailableException: Timeout waiting for database to become available and allow new transactions. Waited 1s. 1 reasons for blocking: Database unavailable.

In [12]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')

In [14]:
# merchant_rank_df=pd.DataFrame()
merchant_56101_community_list=A.copy()
counter=0
for merchant_56101_community in merchant_56101_community_list:
    counter+=1
    print('Doing community {merchant_56101_community}; {counter} of {countlen}'.format(merchant_56101_community=merchant_56101_community, counter=counter,countlen=len(merchant_56101_community_list)))
    print('Drop subgraph:')
    neo4j_query="""CALL gds.graph.drop('merchant_56101_graph') YIELD graphName;"""
    graph.run(neo4j_query).data()
# Create a subgraph of all merchants in merchant_56101_community:
    print('Create subgraph on community {merchant_56101_community}:'.format(merchant_56101_community=merchant_56101_community))
    neo4j_query="""CALL gds.graph.create.cypher(
        'merchant_56101_graph',
        'MATCH (m0:Merchant {{subclass_id:56101}}) 
        WHERE m0.merchant_56101_community = {merchant_56101_community} RETURN id(m0) AS id, 
        m0.totaltransactionamount as totaltransactionamount, m0.totaltransactioncount as totaltransactioncount',
        'MATCH (m0:Merchant)-[rel:MERCHANT_FEET_LINK_56101]->(m1:Merchant) 
        WHERE rel.count > 1 AND 
        m0.merchant_56101_community = {merchant_56101_community} AND 
        m1.merchant_56101_community = {merchant_56101_community}
        RETURN id(m0) AS source, id(m1) AS target, rel.count as count'
    )
    YIELD graphName, nodeCount, relationshipCount, createMillis;
    """.format(merchant_56101_community=merchant_56101_community)
    graph.run(neo4j_query).data()
# Write the MerchantRank on community nodes.  The rank belongs to the community of merchants as defined by the sub graph created above:
    print('MerchantRank community members:')
    neo4j_query="""
    CALL gds.pageRank.write('merchant_56101_graph', {
      maxIterations: 20,
      dampingFactor: 0.85,
      writeProperty: 'merchant_56101_rank',
      relationshipWeightProperty: 'count'
    })
    YIELD nodePropertiesWritten AS writtenProperties, ranIterations;"""
    graph.run(neo4j_query).data()
    print('Extract community members with merchant rank: ')
    neo4j_query="""
    MATCH (m:Merchant {{subclass_id:56101}}) WHERE m.merchant_56101_community = {merchant_56101_community}
    RETURN m.franchisename as franchisename, m.companyname as companyname,  m.merchant_56101_rank as MerchantRank, {merchant_56101_community} as merchant_56101_community 
    ORDER BY MerchantRank DESC;
    """.format(merchant_56101_community=merchant_56101_community)
    df=graph.run(neo4j_query).to_data_frame()

    df['rankID']=range(1,df.shape[0]+1)
    print('Append extracted merchant community to a result dataframe:')    
    merchant_rank_df=merchant_rank_df.append(df,ignore_index=True)
    A.pop(A.index(merchant_56101_community))    
    print('====================================================================================================')

Doing community 1565; 1 of 23
Drop subgraph:
Create subgraph on community 1565:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 2086; 2 of 23
Drop subgraph:
Create subgraph on community 2086:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 1833; 3 of 23
Drop subgraph:
Create subgraph on community 1833:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 1322; 4 of 23
Drop subgraph:
Create subgraph on community 1322:
MerchantRank community members:
Extract community members with merchant rank: 
Append extracted merchant community to a result dataframe:
Doing community 1458; 5 of 23
Drop subgraph:
Create subgraph on community 1458:
MerchantRank community members:
Extract communit

Save the contents of merchant_rank_df into a .csv file:

In [15]:
merchant_rank_df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_rank_df',index=False)

We now have two dataframes thus far.  Dataframe merchant_56101_community_df ontains only the community number and the community count:

In [16]:
merchant_56101_community_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_community_df')

In [17]:
list(merchant_56101_community_df)

['membercount', 'merchant_56101_community']

Dataframe merchant_rank_df contains the community number (merchant_56101_community) and the rank of the merchant (MerchantRank) in that community.  It also has the franchise and company names. Read the same file into merchant_rank_df:

In [18]:
merchant_56101_rank_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_rank_df')

In [19]:
list(merchant_56101_rank_df)

['franchisename',
 'companyname',
 'MerchantRank',
 'merchant_56101_community',
 'rankID']

Merge merchant_56101_rank_df and merchant_56101_community_df on the merchant_56101_community column:

In [20]:
df=merchant_56101_rank_df.merge(merchant_56101_community_df,left_on='merchant_56101_community',
                             right_on='merchant_56101_community',how='left')

In [21]:
df.to_csv('/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_df',index=False)

In [22]:
len(set(df['merchant_56101_community']))

28

In [23]:
df

,franchisename,companyname,MerchantRank,merchant_56101_community,rankID,membercount
0,MCD Ladysmith(,MCD,3.513683,1282,1,27
1,KFC DUNDEE,KFC,3.030635,1282,2,27
2,KFC EZAKHENI,KFC,2.774168,1282,3,27
3,STEERS DUNDE,STEERS,1.936258,1282,4,27
4,Nandos Newcast,NANDOS,1.528236,1282,5,27
...,...,...,...,...,...,...
4945,McDONALDS PO,MCD,0.150000,2940,266,270
4946,BURGER KING KE,BURGER KING,0.150000,2940,267,270
4947,Burger King Kl,BURGER KING,0.150000,2940,268,270
4948,Burger king kl,BURGER KING,0.150000,2940,269,270


### Community aggregation:

Here we use the results of the code above to do aggregation on the fast food communities:

Connect to neo4J:

In [ ]:
graph = Graph("bolt://localhost:7687", user='neo4j', password='newPassword')

In what follows we do two pieces of aggregation
1. By merchant. Include 
    1. community number (merchant_56101_community),
    2. companyname,
    3. franchisename,
    4. merchant rank, 
    5. total transaction amount and 
    6. total number of transactions.    

In [ ]:
neo4j_query="""MATCH (merchant:Merchant {subclass_id:56101})
WHERE EXISTS(merchant.merchant_56101_rank)
RETURN merchant.merchant_56101_community as merchant_56101_community,
merchant.companyname as companyname, 
merchant.franchisename as franchisename, 
merchant.merchant_56101_rank as merchant_rank,
merchant.totaltransactionamount as totaltransactionamount,
merchant.totaltransactioncount as totaltransactioncount
ORDER BY merchant_56101_community ASC;
"""
# merchant_rank_df=merchant_rank_df.append(df,ignore_index=True)
df=graph.run(neo4j_query).to_data_frame()

Create a new column __franchise_rank__.  This column holds the rank of a merchant in a merchant community based on the merchant rank, 1 being the franchise with the highest merchant rank - the franchise most likely to visited by a shopper.

In [ ]:
df['franchise_rank']=df.groupby("merchant_56101_community")["merchant_rank"].rank("dense", ascending=False)

Look at one company and ask the question of how many merchants see this company with the highest merchant rank:

In [ ]:
df[(df.companyname=='BP')&(df.franchise_rank==1)]

In [ ]:
len(set(df.merchant_56101_community))

In [ ]:
df[(df.franchise_rank==1)]

In [ ]:
df[(df.merchant_56101_community==11222)]

Add another two columns, __totaltransactionamount_rank__ is the rank of all franchises in a community base on __totaltransactionamount__ and __totaltransactioncount_rank__ is likewise based on __totaltransactioncount__.

In [ ]:
# df['totaltransactionamount']=-df['totaltransactionamount']
df['totaltransactionamount_rank']=df.groupby("merchant_56101_community")["totaltransactionamount"].rank("dense", ascending=False)
df['totaltransactioncount_rank']=df.groupby("merchant_56101_community")["totaltransactioncount"].rank("dense", ascending=False)

In [ ]:
df[['merchant_56101_community','totaltransactionamount','totaltransactioncount','franchise_rank','totaltransactionamount_rank','totaltransactioncount_rank']]

In [ ]:
df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_community_aggregation_df',index=False)

#### Total client spend by fuel station clients:

In the following aggregation we take all clients who spent at any off the merchant_56101_community fuel stations and aggregate their total spend (at all POS).  We also count the number of distinct clients and merchants belonging to each community.

In [ ]:
neo4j_query="""
MATCH (client:Client)-[t:TRANSACTED_AT]-(merchant:Merchant)
WHERE EXISTS(merchant.merchant_56101_rank)
RETURN merchant.merchant_56101_community as merchant_56101_community,
COUNT(DISTINCT client) as nClients,
SUM(client.totaltransactionamount) as totalspend,
COUNT(DISTINCT merchant) as nMerchants
ORDER BY nClients ASC;
"""
df=graph.run(neo4j_query).to_data_frame()

In [ ]:
df

In [ ]:
min(df['totalspend']/df['nClients'])

Save the results to merchant_56101_community_spend_df:

In [ ]:
df.to_csv(path_or_buf='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_community_spend_df',index=False)

Read the previous two files:

In [ ]:
community_aggregation_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_community_aggregation_df')
community_spend_df=pd.read_csv(filepath_or_buffer='/media/lnr-ai/christo/github_repos/graphing/data/merchant_56101_community_spend_df')

In [ ]:
list(community_aggregation_df)

In [ ]:
list(community_spend_df)

In [ ]:
community_aggregation_df.merge(community_spend_df,how='left',left_on='merchant_56101_community',right_on='merchant_56101_community')

Here we look at totals by community and create new columns __community_totaltransactionamount__ and __community_totaltransactioncount__

In [ ]:
neo4j_query="""MATCH (merchant:Merchant {subclass_id:56101})
WHERE EXISTS(merchant.merchant_56101_rank)
RETURN merchant.merchant_56101_community as merchant_56101_community,
COUNT(merchant.merchant_56101_community) as count_merchant_56101_community,
SUM(merchant.totaltransactionamount) as community_totaltransactionamount,
SUM(merchant.totaltransactioncount) as community_totaltransactioncount
ORDER BY merchant_56101_community ASC;
"""
df=graph.run(neo4j_query).to_data_frame()

In [ ]:
df